In [40]:
from openpyxl import load_workbook
import pandas as pd 
import numpy as np
from itertools import islice
from openpyxl.utils import FORMULAE


In [38]:
# wb = load_workbook('sample.xlsx')

wb = load_workbook('sample.xlsx')
ws = wb['Sheet1']

#create df of plate map
samp = ws.values
columns = next(samp)[1:]
samp = list(samp)
idx = [r[0] for r in samp]
samp = (islice(r, 1, None) for r in samp)
df = pd.DataFrame(samp, index=idx, columns=columns)

#create df of raw data values
ws2 = wb['Sheet2']

data = ws2.values
columns = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df2 = pd.DataFrame(data, index=idx, columns=columns)

print(df, '\n', df2)

    1    2    3    4    5    6    7    8    9     10    11    12
A   S1   S2   S3   S4   S5   S6   S7   S8   S9   S10   S11   S12
B   S1   S2   S3   S4   S5   S6   S7   S8   S9   S10   S11   S12
C  S13  S14  S15  S16  S17  S18  S19  S20  S21   S22   S23   S24
D  S13  S14  S15  S16  S17  S18  S19  S20  S21   S22   S23   S24
E  S25  S26  S27  S28  S29  S30  S31  S32  S33   S34   S35   S36
F  S25  S26  S27  S28  S29  S30  S31  S32  S33   S34   S35   S36
G    A    B    C    D    E    F    G    H    I  None  None  None
H    A    B    C    D    E    F    G    H    I  None  None  None 
        1      2       3       4      5       6      7       8       9     10  \
A  0.7000  0.600  0.4000  0.2000  0.600  0.1000  0.990  0.7000  0.9300  0.87   
B  0.8000  0.930  0.4300  0.2400  0.930  0.1000  1.030  0.8900  0.8100  0.39   
C  1.2000  1.600  1.3000  1.6000  1.300  1.9000  1.300  1.7000  1.3300  1.50   
D  1.5000  1.200  1.4000  1.6000  1.000  1.8000  1.400  1.2000  1.5000  1.20   
E     NaN    

In [39]:
#find position of specific value in df

def findPosInDf(dfIn,findme):
    positions = []
    irow =0
    while ( irow < len(dfIn.index)):
        list_colPositions=dfIn.columns[dfIn.iloc[irow,:]==findme].tolist()   
        if list_colPositions != []:
            colu_iloc = dfIn.columns.get_loc(list_colPositions[0])
            positions.append([irow  , colu_iloc ])
        irow +=1

    return positions


#print(findPosInDf(df, 'I'))

bgPos = findPosInDf(df, 'I')

#loop to give value at every position of 'I'
for x in bgPos:  
    print(df2.iat[x[0],x[1]])
    





0.0945
0.0983
